# Quantum `conda` environment

Let's create a new `conda` environment called "q-env" and running Python 3.10:

```bash
conda create -n q-env python=3.10 -y
```

After the activation

```bash
conda activate q-env
```

start by upgrading `pip` and `ipykernel` via:

```bash
pip install --upgrade pip ipykernel
```

Continue by installing TensorFlow 2.14 (with cuDNN 8.7 and CUDA 11.8):

```bash
pip install tensorflow[and-cuda]==2.14
```

Test that the installation succeeds and that TensorFlow correctly accesses the GPU by executing what follows:

In [1]:
import tensorflow as tf

print(f"[DEBUG] TensorFlow version {tf.__version__}")

devices = tf.config.list_physical_devices("GPU")
if len(devices) > 0:
    rnd = tf.random.uniform(shape=(100, 1))
    print("[STATUS] GPU available")
else:
    print("[STATUS] GPU not available") 

2024-06-10 20:52:49.464568: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 20:52:49.464624: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 20:52:49.464648: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-10 20:52:49.472189: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 20:52:51.399218: W tensorflow/compiler/

[DEBUG] TensorFlow version 2.14.0
[STATUS] GPU not available


2024-06-10 20:52:54.379316: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-10 20:52:54.427821: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Continue by installing PyTorch 2.1 based on CUDA 11.8:

```bash
pip install torch==2.1 --index-url https://download.pytorch.org/whl/cu118
```

Test that the installation succeeds and that PyTorch correctly accesses the GPU by executing what follows:

In [2]:
import torch

print(f"[DEBUG] PyTorch version {torch.__version__}")

torch.cuda.init()

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
    rnd = torch.randn(size=(100, 1)).to(device)
    print("[STATUS] GPU available")
else:
    print("[STATUS] GPU not available")

[DEBUG] PyTorch version 2.1.0+cu118
[STATUS] GPU available


Continue by installing Jax based on CUDA 11.8 (locally preinstalled):

```bash
pip install --upgrade "jax[cuda11_pip]==0.4.25" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
```

Test that the installation succeeds and that Jax correctly accesses the GPU by executing what follows:

In [3]:
import jax

print(f"[DEBUG] Jax version {jax.__version__}")

def jax_has_gpu():
    try:
        _ = jax.device_put(jax.numpy.ones(1), device=jax.devices('gpu')[0])
        return True
    except:
        return False

if jax_has_gpu():
    print("[STATUS] GPU available")
else:
    print("[STATUS] GPU not available")

[DEBUG] Jax version 0.4.25
[STATUS] GPU available


Continue by installing Pennylane Lightning with GPU-acceleration:

```bash
pip install --upgrade pennylane pennylane-lightning pennylane-lightning-gpu
```

Once installed, you can try to enable the CUDA 11 variant of the NVIDIA cuQuantum custatevec library via `pip`:

```bash
pip install nvidia-cusparse-cu11 nvidia-cublas-cu11 nvidia-cuda-runtime-cu11 custatevec-cu11
```

Test that the installation succeeds and that Pennylane Lightning correctly accesses the GPU by executing what follows:

In [4]:
import pennylane as qml

print(f"[DEBUG] PennyLane version {qml.__version__}")

dev = qml.device("lightning.gpu", wires=2)

@qml.qnode(dev)
def circuit():
  qml.Hadamard(wires=0)
  qml.CNOT(wires=[0,1])
  return qml.expval(qml.PauliZ(0))

circuit()

[DEBUG] PennyLane version 0.36.0


/envs/q-cu11.8/lib/python3.10/site-packages/pennylane_lightning/lightning_gpu/lightning_gpu.py:94: UserWarning: libcudart.so.12: cannot open shared object file: No such file or directory
  warn(str(e), UserWarning)


ImportError: Pre-compiled binaries for lightning.gpu are not available. To manually compile from source, follow the instructions at https://pennylane-lightning.readthedocs.io/en/latest/installation.html.

Continue by installing Qiskit Aer with GPU-acceleration (based on CUDA 11):

```bash
pip install qiskit qiskit-aer-gpu-cu11
```

Test that the installation succeeds and that Qiskit Aer correctly accesses the GPU by executing what follows:

In [5]:
import qiskit
from qiskit_aer import AerSimulator

print(f"[DEBUG] Qiskit version {qiskit.__version__}")

# Generate 3-qubit GHZ state
circ = qiskit.QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(1, 2)
circ.measure_all()

# Construct an ideal simulator
aersim = AerSimulator()

# Perform an ideal simulation
result_ideal = aersim.run(circ).result()
counts_ideal = result_ideal.get_counts(0)
print('[DEBUG] Counts(ideal):', counts_ideal)
# Counts(ideal): {'000': 493, '111': 531}

[DEBUG] Qiskit version 1.0.2
[DEBUG] Counts(ideal): {'000': 488, '111': 536}


Finally, continue by installing Qibo, QiboJIT, and CuPy via `conda`:

```bash
conda install qibo qibojit cupy
```

In [6]:
import cupy as cp

print(f"[DEBUG] CuPy version {cp.__version__}")

n_cuda_devices = cp.cuda.runtime.getDeviceCount()

if n_cuda_devices > 0:
    x = cp.array([1, 2, 3])
    x.device
    print("[STATUS] GPU available")
else:
    print("[STATUS] GPU not available")

[DEBUG] CuPy version 13.0.0
[STATUS] GPU available
